<a href="https://colab.research.google.com/github/arnisafazla/CS490/blob/main/CS490_download_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code will download the MoCap files from the Emotional Body Motion database (provided by the Max-Planck Institute for Biological Cybernetics in Tuebingen, Germany), to the specified folder in a structured way.

It will create a folder called dataset, and subfolders for each specified emotion, and will download the MoCap files to their corresponding folders.


In [ ]:
destination_path = # PATH-TO-THE-FOLDER-OF-THE-DATASET
# enter already filtered url
dataset_path = 'http://ebmdb.tuebingen.mpg.de/'
emotions = ["anger", "joy", "fear", "neutral", "sadness", "surprise", "shame", "pride"]

In [ ]:
# take json file and get the list elements
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys, os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def make_titles(text):
  """Remove punctuation, make it lowercase, replace space with underscore"""
  clean = re.sub('[^\w\s]', '', text)
  clean = re.sub('\s+', '_', clean)
  if clean[0] != '_':
    clean = '_' + clean
  return clean

In [ ]:
def download_emotions(emotions, destination_path):
  main = os.path.join(destination_path, 'dataset')
  if not os.path.exists(main):
    os.mkdir(main)
  for emotion in emotions:
    emotion_path = os.path.join(main, emotion)
    if not os.path.exists(emotion_path):
      os.mkdir(emotion_path)
    filtered_url = "http://ebmdb.tuebingen.mpg.de/index.php?set_filters=Search&filter_intended_emotion={}&filter_intended_polarity=&filter_modal_category=&filter_modal_polarity=&filter_accurate_category=one&filter_accurate_polarity=one&filter_acting_task=&filter_acting_subtask=&filter_actor=&filter_gender=&filter_handedness=&filter_native_tongue=&filter_responses=&filter_text=".format(emotion)
    r = requests.get(filtered_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    table = soup.find('table', attrs = {'id':'motion_table'})
    i = 0
    for row in table.findAll('tr'):
      link_tds = row.findAll('td', attrs = {'id':'td_download_links'})
      if link_tds != []:
        i += 1
        explanation = row.findAll('td')[-1].string
        title = str(i) + (make_titles(explanation))
        if len(title) > 50:
          title = title[0:50]
        title = title + '.bvh'
        for a in link_tds[0].find_all('a', href=True):
          bvh1 = a['href']
          bvh2 = dataset_path + bvh1[2:]
          if 'bvh' in bvh2:
            resp = requests.get(bvh2)
            with open( title, "wb" ) as file :
              file.write( resp.content )
            ! mv $title emotion_path
            print(title)

In [ ]:
download_emotions(emotions, destination_path)